In [1]:
from sklearn.model_selection import train_test_split # importa método train_test_split. Existem outros como: K-Folds cross-validator
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [ ]:
!wget https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/diabetes.csv

dados = pd.read_csv("diabetes.csv")

# Análise exploratória dos dados

https://minerandodados.com.br/analise-exploratoria-de-dados-passo-a-passo-com-python/

<br>Analisar/Selecionar Features/Campos, colunas, atributos - 
<br>como selecionar - https://minerandodados.com.br/aprenda-como-selecionar-features-para-seu-modelo-de-machine-learning/
<br> Algumas Features (Features não informativas) pode adicionar ruído ao modelo

In [3]:
dados.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
dados.shape

(768, 9)

In [ ]:
dados["Glucose"]

In [ ]:
dados.info()

In [ ]:
dados["Glucose"].unique()

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.value_counts.html
# Retorne uma série contendo contagens de linhas exclusivas no DataFrame.
dados.Glucose.value_counts()

In [ ]:
dados2 = dados[dados.Outcome ==1]  #mostra todos os registros cujo o OutCome seja igual 1
#dados2 = dados[dados.Outcome ==1].sample(5)
dados2

In [ ]:
# verifica se existem campos/features/atributos vazios - checando Missing Values
dados2.isnull().sum()

In [ ]:
#https://datatofish.com/replace-nan-values-with-zeros/
# Preencher as colunas NaN - não definida
# ou .fill('Yes') ou .fillna(data['colunax'].mean())
dados['Outcome'] = dados['Outcome'].fillna('0') 

In [ ]:
# chegar novamente Missing Value
dados.isnull().sum()

#Transformandos dados categóricos

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
#colunax_values = {'Female':0,'Male':1}
#coluna_casado_values = {'No':0,'Yes':1}
# dados.replace({'colunax':colunax_values, 'coluna_casado':coluna_casado_values}, inplace=True)
# exemplo
## OutCome_values = {'Yes':1,'No':0}
## dados2.replace({'OutCome': OutCome_values},inplace=True)

# Dividir o conjunto de dados em duas variáveis X e y

In [12]:

# X mantém os dados que a árvore de decisão vai usar para aprender e y são os rótulos.
features = dados.drop(["Outcome"], axis=1) #Excluir a coluna de resultado
X = np.array(features)  # dados
y = np.array(dados["Outcome"])  #rótulos

In [13]:
features.columns # verificar que a coluna Outcome foi excluída

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')

# Dividir dos dados para Treinamento e Testes/Validação

In [14]:
# Divide o conjunto de dados com a seguinte função scikit-learn
# Usamos 20% (0.20) do conjunto de dados para construir o conjunto de validação.
# retorn uma tupla
# random_state=0 = Seed
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, test_size=0.20)

#Selecionar o melhor classificado através de Pipeline e GridSearchCV

In [51]:
# Classificador Árvore de decisão
arvore = DecisionTreeClassifier(min_samples_leaf=10, max_depth=8, min_samples_split=50)

clf = arvore.fit(X_train, y_train) # método fit faz o treinamento do modelo
clf

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=50,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
from sklearn import tree
tree.plot_tree(clf)  # Existem outros modelos de apresentação

In [53]:
arvore.tree_.max_depth

7

In [54]:
# Usaremos o conjunto de validação para medir o desempenho do modelo
validation_prediction = arvore.predict(X_val)
training_prediction   = arvore.predict(X_train)

# from sklearn.metrics import accuracy_score
print('Accuracy training set: ', accuracy_score(y_true=y_train, y_pred=training_prediction))
print('Accuracy validation set: ', accuracy_score(y_true=y_val, y_pred=validation_prediction))

Accuracy training set:  0.8268156424581006
Accuracy validation set:  0.7359307359307359


In [55]:
print(metrics.classification_report(y_val,clf.predict(X_val)))

              precision    recall  f1-score   support

           0       0.79      0.81      0.80       151
           1       0.62      0.60      0.61        80

    accuracy                           0.74       231
   macro avg       0.71      0.70      0.71       231
weighted avg       0.73      0.74      0.73       231



# Persistindo o Modelo em disco

In [ ]:
from sklearn.externals import  joblib

In [21]:
joblib.dump( arvore,'modelo.pkl'  )   #tira da Ram e salva no disco- joblib.dump

['modelo.pkl']

In [22]:
!ls


diabetes.csv  modelo.pkl  sample_data


In [23]:
import pickle
with open('modelo2.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [24]:
!ls

diabetes.csv  modelo2.pkl  modelo.pkl  sample_data


In [25]:
# carregando o modelo do disco para a memória usando a ferramenta joblib
model = joblib.load('modelo.pkl')

In [56]:
# carregando o modelo do disco para a memória com pickle
with open('modelo2.pkl', 'rb') as f:
    model2 = pickle.load(f)

In [35]:
# Verificando os atributos do modelo

print("Atributos do modelo:\n\nClasses:{}\n\nParamestros:{}".format(model.classes_,model.get_params))

Atributos do modelo:

Classes:[0 1]

Paramestros:<bound method BaseEstimator.get_params of DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=50,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')>


In [57]:
# Teste de Classificação - Após treinamento e gerado o modelo

meus_dados = np.array([[6,148,72,35,0,33.6,0.627,50]])
model.predict(meus_dados)

array([1])

In [45]:
#Probabilidade de Classes

model.predict_proba(meus_dados)

#Obs- mostra a probabilidade de ser 13% para classe 0 e 86% para a classe 1

array([[0.13888889, 0.86111111]])

In [58]:
model2.predict_proba(meus_dados)

array([[0.13888889, 0.86111111]])

# Flask

In [ ]:
! pip install flask-ngrok

In [27]:
from flask_ngrok import run_with_ngrok 
from flask import Flask

app = Flask(__name__)
run_with_ngrok (app) #starts ngrok quando o aplicativo é executado
@ app.route ("/") 
def home (): 
    return "<h1> Executando Flask no Google Colab! </h1>" 
  

In [ ]:
app.run ()

....




# Outro classificador

##Selecionando o classificado RandomForest

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
#n_estimator= 100 - gerar 100 árvores
clf_rf = RandomForestClassifier(n_estimators=100, min_samples_split=2)

In [31]:
# Divide o conjunto de dados com a seguinte função scikit-learn
# Usamos 20% (0.20) do conjunto de dados para construir o conjunto de validação.
# retorn uma tupla
# random_state=0 = Seed
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, test_size=0.30)

In [32]:
clf_rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#Métricas de Validação

In [43]:
from sklearn import  metrics
from sklearn.model_selection import  train_test_split

In [34]:
# Métrica nos dados de Validação
# pd.crosstab = tabela cruzada de dois ou mais fatores
cross_table = pd.crosstab(y_train, clf_rf.predict(X_train), rownames=['Actual'], colnames=['Predicted'], margins=True)

In [35]:
cross_table

# Observer que acertou 100% - teve um overfitting - decorou os dados

Predicted,0,1,All
Actual,,,
0,349,0,349
1,0,188,188
All,349,188,537


In [36]:
# # Métrica nos dados de testes
cross_table = pd.crosstab(y_val, clf_rf.predict(X_val), rownames=['Actual'], colnames=['Predicted'], margins=True)
cross_table

# Analisando:  de 99 acertou 78 errou 21; de 55 acertou 36 e errou 19

Predicted,0,1,All
Actual,,,
0,120,31,151
1,28,52,80
All,148,83,231


In [37]:
print(metrics.classification_report(y_val,clf_rf.predict(X_val)))

              precision    recall  f1-score   support

           0       0.81      0.79      0.80       151
           1       0.63      0.65      0.64        80

    accuracy                           0.74       231
   macro avg       0.72      0.72      0.72       231
weighted avg       0.75      0.74      0.75       231



In [38]:
# Usaremos o conjunto de validação para medir o desempenho do modelo
validation_prediction = clf_rf.predict(X_val)
training_prediction   = clf_rf.predict(X_train)

# from sklearn.metrics import accuracy_score
print('Accuracy training set: ', accuracy_score(y_true=y_train, y_pred=training_prediction))
print('Accuracy validation set: ', accuracy_score(y_true=y_val, y_pred=validation_prediction))

Accuracy training set:  1.0
Accuracy validation set:  0.7445887445887446
